# **Yahoo Finance Data**

# 1. OOP Lähenemine andmete hankimiseks ja töötlemiseks
Klassid ja objektid:

Luuakse klassid andmete hankimiseks, töötlemiseks ja analüüsimiseks.
Näites kasutatakse Yahoo Finance'i API-t aktsiahindade ja finantsinfo saamiseks.

Selgitused:
StockData klass: Selle klassi abil saab aktsiahindade ja finantsinfo päringuid Yahoo Finance API-st. get_history, get_financials, ja get_info meetodid pakuvad erinevaid andmeallikaid.

In [1]:
import yfinance as yf

class StockData:
    def __init__(self, symbol: str):
        self.symbol = symbol
        self.stock = yf.Ticker(symbol)

    def get_history(self, period="1mo"):
        return self.stock.history(period=period)

    def get_financials(self):
        return self.stock.financials

    def get_info(self):
        return self.stock.info

    def __str__(self):
        return f"StockData for {self.symbol}"

# Kasutamine
apple = StockData("AAPL")
print(apple.get_history())
print(apple.get_financials())
print(apple.get_info())


                                 Open        High         Low       Close  \
Date                                                                        
2024-07-30 00:00:00-04:00  218.936591  220.075273  215.870133  218.547043   
2024-07-31 00:00:00-04:00  221.183983  223.561236  220.374922  221.823242   
2024-08-01 00:00:00-04:00  224.110590  224.220463  216.769097  218.107544   
2024-08-02 00:00:00-04:00  218.896619  225.339173  217.458296  219.605804   
2024-08-05 00:00:00-04:00  198.859822  213.253166  195.773398  209.028061   
2024-08-06 00:00:00-04:00  205.062641  209.747221  200.837536  206.990402   
2024-08-07 00:00:00-04:00  206.660786  213.392999  206.151381  209.577423   
2024-08-08 00:00:00-04:00  212.863619  213.952355  208.588569  213.063385   
2024-08-09 00:00:00-04:00  211.854792  216.529374  211.724938  215.990005   
2024-08-12 00:00:00-04:00  216.070007  219.509995  215.600006  217.529999   
2024-08-13 00:00:00-04:00  219.009995  221.889999  219.009995  221.270004   

# 2. Pydantic: Andmete valideerimine
Näites kasutatakse Pydanticut, et tagada õigete sisendandmete vorming.

Selgitused:
StockQuery klass: Pydanticu andmete valideerimisklass, mis tagab, et sümbol pole tühi ja ajaperiood on kehtiv.

validator: Funktsioonid, mis kontrollivad sisendandmete õigsust enne edasist töötlemist.

In [2]:
from pydantic import BaseModel, validator

class StockQuery(BaseModel):
    symbol: str
    period: str = "1mo"

    @validator('symbol')
    def symbol_must_not_be_empty(cls, v):
        if not v:
            raise ValueError("Symbol must not be empty")
        return v

    @validator('period')
    def valid_period(cls, v):
        allowed_periods = ["1d", "5d", "1mo", "3mo", "6mo", "1y", "5y", "10y", "ytd", "max"]
        if v not in allowed_periods:
            raise ValueError(f"Period must be one of {allowed_periods}")
        return v

# Kasutamine
try:
    query = StockQuery(symbol="AAPL", period="5y")
    print(f"Query: {query.symbol}, Period: {query.period}")
except ValueError as e:
    print(f"Error: {e}")


<ipython-input-2-05d9b16fae23>:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('symbol')
<ipython-input-2-05d9b16fae23>:13: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('period')


Query: AAPL, Period: 5y


# 3. Design Patterns: Factory Pattern
Kasutame Factory Design Pattern'i, et luua erinevaid aktsiate andmeallikaid sõltuvalt kasutaja vajadustest.

Selgitused:

StockFactory klass: Factory Pattern'i näide, mis loob Yahoo ja AlphaVantage andmeallikaid vastavalt kasutaja soovile.

YahooStockData ja AlphaVantageStockData klassid: Need klassid on spetsiifilised andmeallikate jaoks ja pärinevad põhiklassist StockData.

In [3]:
class StockFactory:
    def create_stock_data(self, source: str, symbol: str):
        if source == "Yahoo":
            return YahooStockData(symbol)
        elif source == "AlphaVantage":
            return AlphaVantageStockData(symbol)
        else:
            raise ValueError(f"Unknown source: {source}")

class YahooStockData(StockData):
    def __init__(self, symbol: str):
        super().__init__(symbol)

class AlphaVantageStockData(StockData):
    def __init__(self, symbol: str):
        super().__init__(symbol)
        # additional setup for AlphaVantage API

# Kasutamine
factory = StockFactory()
yahoo_stock = factory.create_stock_data("Yahoo", "AAPL")
print(yahoo_stock.get_history())


                                 Open        High         Low       Close  \
Date                                                                        
2024-07-30 00:00:00-04:00  218.936591  220.075273  215.870133  218.547043   
2024-07-31 00:00:00-04:00  221.183983  223.561236  220.374922  221.823242   
2024-08-01 00:00:00-04:00  224.110590  224.220463  216.769097  218.107544   
2024-08-02 00:00:00-04:00  218.896619  225.339173  217.458296  219.605804   
2024-08-05 00:00:00-04:00  198.859822  213.253166  195.773398  209.028061   
2024-08-06 00:00:00-04:00  205.062641  209.747221  200.837536  206.990402   
2024-08-07 00:00:00-04:00  206.660786  213.392999  206.151381  209.577423   
2024-08-08 00:00:00-04:00  212.863619  213.952355  208.588569  213.063385   
2024-08-09 00:00:00-04:00  211.854792  216.529374  211.724938  215.990005   
2024-08-12 00:00:00-04:00  216.070007  219.509995  215.600006  217.529999   
2024-08-13 00:00:00-04:00  219.009995  221.889999  219.009995  221.270004   

# 4. Decorators: Lisame funktsionaalsust päringutele
Decorators on kasulikud, kui soovime lisada funktsionaalsust ilma olemasolevat koodi muutmata.

Selgitused:
cache_results decorator: See salvestab funktsiooni tulemused, et vältida korduvaid päringuid ja kiirendada koodi käivitamist.
CachedStockData klass: See klass laiendab StockData klassi ja lisab cache funktsionaalsuse Yahoo Finance API päringutele.

In [4]:
import functools

def cache_results(func):
    cache = {}

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if args not in cache:
            cache[args] = func(*args, **kwargs)
        return cache[args]

    return wrapper

class CachedStockData(StockData):
    @cache_results
    def get_history(self, period="1mo"):
        return super().get_history(period=period)

# Kasutamine
cached_apple = CachedStockData("AAPL")
print(cached_apple.get_history())


                                 Open        High         Low       Close  \
Date                                                                        
2024-07-30 00:00:00-04:00  218.936591  220.075273  215.870133  218.547043   
2024-07-31 00:00:00-04:00  221.183983  223.561236  220.374922  221.823242   
2024-08-01 00:00:00-04:00  224.110590  224.220463  216.769097  218.107544   
2024-08-02 00:00:00-04:00  218.896619  225.339173  217.458296  219.605804   
2024-08-05 00:00:00-04:00  198.859822  213.253166  195.773398  209.028061   
2024-08-06 00:00:00-04:00  205.062641  209.747221  200.837536  206.990402   
2024-08-07 00:00:00-04:00  206.660786  213.392999  206.151381  209.577423   
2024-08-08 00:00:00-04:00  212.863619  213.952355  208.588569  213.063385   
2024-08-09 00:00:00-04:00  211.854792  216.529374  211.724938  215.990005   
2024-08-12 00:00:00-04:00  216.070007  219.509995  215.600006  217.529999   
2024-08-13 00:00:00-04:00  219.009995  221.889999  219.009995  221.270004   

# 5. Näide: Kogu süsteem koos
Lõpuks ühendame kõik eelnevad tehnikad ühte terviklikku lahendusse.

Selgitused:

StockQuery klass: Pydanticu abil tagame korrektse sümboli ja perioodi.

StockData klass: Vastutab andmete hankimise eest.

StockAnalyzer klass: Teeb analüüsi (näiteks liikuv keskmine).

cache_results decorator: Kiirendab päringuid, hoides andmeid vahemälus.

In [8]:
import yfinance as yf
from pydantic import BaseModel
from functools import wraps
import pandas as pd

# Cache decorator for optimization
def cache_results(func):
    cache = {}

    @wraps(func)
    def wrapper(*args, **kwargs):
        if args not in cache:
            cache[args] = func(*args, **kwargs)
        return cache[args]

    return wrapper

# Pydantic model for query validation
class StockQuery(BaseModel):
    symbol: str
    periods: list = ["1mo", "3mo", "6mo", "1y"]  # Different periods to try

# Class for fetching stock data from Yahoo Finance
class StockData:
    def __init__(self, symbol: str):
        self.symbol = symbol
        self.stock = yf.Ticker(symbol)

    @cache_results
    def get_history(self, period="1mo"):
        history = self.stock.history(period=period)
        if history.empty:
            raise ValueError(f"No data found for {self.symbol} in the specified period.")
        return history

    def get_info(self):
        return self.stock.info

# Analyzer class to perform data analysis on stock data
class StockAnalyzer:
    def __init__(self, data: StockData):
        self.data = data

    def moving_average(self, window: int, periods: list):
        for period in periods:
            try:
                hist = self.data.get_history(period)
                if hist.empty:
                    continue

                # Ensure there's enough data to compute the moving average
                if len(hist) < window:
                    print(f"Not enough data for period {period}.")
                    continue

                ma = hist['Close'].rolling(window=window).mean()
                return period, ma.dropna()  # Drop NaN values to focus on valid results
            except ValueError as e:
                print(f"Error fetching data for period {period}: {e}")
                continue

        raise ValueError("Unable to compute moving average with provided periods.")

# Example usage
try:
    query = StockQuery(symbol="AAPL")
    stock_data = StockData(query.symbol)
    analyzer = StockAnalyzer(stock_data)

    period, moving_avg = analyzer.moving_average(30, query.periods)
    print(f"Moving Average (30 days) using period {period}:")
    print(moving_avg)
except ValueError as e:
    print(f"Error: {e}")


Not enough data for period 1mo.
Moving Average (30 days) using period 3mo:
Date
2024-07-12 00:00:00-04:00    210.923856
2024-07-15 00:00:00-04:00    212.359195
2024-07-16 00:00:00-04:00    213.776555
2024-07-17 00:00:00-04:00    214.936879
2024-07-18 00:00:00-04:00    215.930062
2024-07-19 00:00:00-04:00    216.876966
2024-07-22 00:00:00-04:00    217.858498
2024-07-23 00:00:00-04:00    218.794747
2024-07-24 00:00:00-04:00    219.641101
2024-07-25 00:00:00-04:00    219.985369
2024-07-26 00:00:00-04:00    220.148181
2024-07-29 00:00:00-04:00    220.281360
2024-07-30 00:00:00-04:00    220.491450
2024-07-31 00:00:00-04:00    220.671575
2024-08-01 00:00:00-04:00    220.807085
2024-08-02 00:00:00-04:00    221.146026
2024-08-05 00:00:00-04:00    221.205291
2024-08-06 00:00:00-04:00    221.174992
2024-08-07 00:00:00-04:00    221.199963
2024-08-08 00:00:00-04:00    221.201961
2024-08-09 00:00:00-04:00    221.273212
2024-08-12 00:00:00-04:00    221.511663
2024-08-13 00:00:00-04:00    221.670683
